Read drive table from amazon.db

In [1]:
import sqlite3

import pandas as pd


db = sqlite3.connect('../data/amazon.db')

drive_df = pd.read_sql('select * from drive', db)
drive_df.head()

,asin,catagory,manufacturer,brand,model_name,product_name,drive_type,drive_subtype,capacity,last_update
0,B01K8A29BE,drive,ADATA USA,ADATA,su800,ADATA SU800 128GB 3D-NAND 2.5 Inch SATA III Hi...,ssd,sata,128gb,5722242
1,B01K8A2A0E,drive,ADATA USA,ADATA,su800,ADATA SU800 256GB 3D-NAND 2.5 Inch SATA III Hi...,ssd,sata,512gb,5722182
2,B01K8A29E6,drive,ADATA USA,ADATA,su800,ADATA USA Ultimate Su800 1TB 3D Nand 2.5 Inch ...,ssd,sata,1tb,5722216
3,B07GBRG2G8,drive,ADATA,ADATA,su800,ADATA SU800 2TB 3D-NAND 2.5 Inch SATA III High...,ssd,sata,2tb,5722242
4,B07TDW8Z3M,drive,ADATA,ADATA,su760,ADATA SU760 256GB 3D NAND 2.5 Inch SATA III In...,ssd,sata,256gb,5722242


Query all time series

In [2]:
def search(asin):
    drive_ts = pd.read_sql_query(f'select keepa_minute, value1 from drive_ts where asin = "{asin}" and keepa_idx=11 and value1 != -1', db)
    drive_ts['ts'] = pd.to_datetime((drive_ts['keepa_minute'] + 21564000) * 60, unit='s')
    drive_ts.set_index('ts', inplace=True)
    drive_ts = drive_ts.resample('1D').ffill()
    drive_ts = drive_ts.resample('M').mean()
    if '2021-01' not in drive_ts.index:
        return None
    base_stock = drive_ts.loc['2021-01']['value1'][0]
    price_dict = {'base_stock': base_stock}
    for d in ['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10']:
        if d in drive_ts.index:
            price_dict[d] = drive_ts.loc[d]['value1'][0] - base_stock
        else:
            price_dict[d] = 1
    return price_dict

print(search(drive_df['asin'][15]))

{'base_stock': 5.709677419354839, '2021-01': 0.0, '2021-02': 0.8617511520737322, '2021-03': 0.6774193548387091, '2021-04': -0.6430107526881725, '2021-05': 2.645161290322581, '2021-06': -0.5763440860215052, '2021-07': -0.67741935483871, '2021-08': -0.774193548387097, '2021-09': -1.7430107526881722, '2021-10': -1.774193548387097}


In [3]:
df_dict = []

for idx, row in drive_df.iterrows():
    ts = search(row['asin'])
    if ts is None:
        continue
    ts['asin'] = row['asin']
    ts['brand'] = row['brand']
    ts['product'] = row['model_name']
    ts['drive_type'] = row['drive_type']
    ts['drive_subtype'] = row['drive_subtype']
    ts['capacity'] = row['capacity']
    df_dict.append(ts)


df = pd.DataFrame(df_dict)
df.insert(0, 'CHIA_IMPACT', [0] * len(df))
len(df)

288

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=60, random_state=501)
df_train.to_csv('drive_stock_train.csv', index=False)
df_test.to_csv('drive_stock_test.csv', index=False)

Close database connection.

In [5]:
db.close()